In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

In [3]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [4]:
# #Replacing Null values of Company_type to Other
data['company_type'].fillna('Other', inplace = True)
test['company_type'].fillna('Other', inplace = True)

# #Replacing NULL values of Company Size with 0
data['company_size'].fillna(0, inplace  = True)
test['company_size'].fillna(0, inplace  = True)

# #Managing all values accordingly
data.experience[data.experience == '>20'] = 21 
data.experience[data.experience == '<1'] = 0 

test.experience[test.experience == '>20'] = 21 
test.experience[test.experience == '<1'] = 0 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(data['last_new_job'].mode()[0], inplace = True)
test['last_new_job'].fillna(test['last_new_job'].mode()[0], inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna(data['education_level'].mode()[0], inplace = True)
test['education_level'].fillna(test['education_level'].mode()[0], inplace = True)

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna(data['enrolled_university'].mode()[0], inplace = True)
test['enrolled_university'].fillna(test['enrolled_university'].mode()[0], inplace = True)

# #Filling NULL values with mode
data.gender.fillna(data['gender'].mode()[0], inplace =  True)
test.gender.fillna(test['gender'].mode()[0], inplace =  True)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [5]:
X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

In [6]:
from sklearn import preprocessing

for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))

# Gradient Boosting

In [108]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection  import GridSearchCV

In [168]:
est = GradientBoostingClassifier(n_estimators = 1800, learning_rate=0.01, max_depth = 3,
                                 min_samples_leaf = 10, max_features = 0.25)

In [169]:
est.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=4,
              max_features=0.25, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=10, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1750,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [170]:
preds = est.predict_proba(X_test)

In [171]:
preds[:,1]

array([0.76602361, 0.06357999, 0.37968663, ..., 0.20364144, 0.25471276,
       0.33127922])

In [172]:
sub['target'] = preds[:,1]

In [173]:
subd = sub.to_csv('1.csv', index = False)

# LGBM

In [7]:
#Main Model
import lightgbm as lgb


parameters = {
    #'objective': 'multiclass',
    #'num_class':5,
    'metric': 'auc',
    #'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'categorical_feature': 5,
    'verbose': 0
}
#For Light gbm the trainig data should in form of Dataset
#Converting the X_train to the lgb Dataset
train_data = lgb.Dataset(X, label=y)
#test_data = lgb.Dataset(X_val, label = y_val)

model = lgb.train(parameters,
                       train_data,
                      
                       num_boost_round=65,
                       #early_stopping_rounds=100
                 )

#y = model.predict(X_test)

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\lightgbm\basic.py:657: UserWarning: categorical_feature keyword has been found in `params` and will be ignored. Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


In [8]:
preds = model.predict(X_test)

In [9]:
sub['target'] = preds

In [10]:
subd = sub.to_csv('1.csv', index = False)

# Grid Search CV